# Recommender System 

## Understanding Reccomender Systems

### Content Based Filtering
        - Uses item features to provide recommendations (uses the features of a certain item to recommend other items with simular features)
        - A great example is if a user watches a certain video from a certain set of traits, we can reccomend other videos based on those traits
        - A limitation with content-based filtering is that it only leverages item simularities

### Collaborative Filtering
        - Uses simularities between items and users simultaneously to provide reccomendations (gives reccomendations to User A based on simular interests of user B)
        - Explicit Feedback: user giving direct feedback (ratings, comments, etc.)
        - Implicit FeedBack: more suttle, your indirect behavior towards an item (watch-time, click-rate, etc.)
        - An example could be that user 1 has watched movie A, B, and C but user 2 has only watched movie A and C, we can use info based off of user A and reccomend to user B 
          that they watch movie B. 
        - Collaborative Filtering, (too me) is a better route to go

#### Collaborative Filtering in practice
        - We can assign a values between -1 and 1 to users for interest in certain movies, -1 means the most interest 1 means the least, and we can do the same for movies
          -1 means it's more of a certain interest of the user and 1 means it's not in the interest of the user
        - In this example we hand-engineered the one-dimensional embeddings, in practice, these embeddings are much higher in dimensions, but we learn these embeddings automatically. 
          that's the beauty of collborative filtering models. 
        -  U is the user embeddings and V is the product embeddngs, the product of these 2 is A, which is a predictive feedback matrix.
        - Our optimization objectie is to minimize the summation of the squared difference between the feedback labels and the predicted feedback
        - We can solve this using SGD or Weighted Alternative Least Squares (WALS), WALS is specific to this problem
        - The idea of WALS is that for each iteration we alternate between fix U and solve for V, and then fixing V and solving for U. 
        - WALS usually converges much faster than SGD, but SGD is more flexible with other loss functions
        - We have only talked about observed items, but we still have the un-observed ones


##### Un-Observed Items in collaborative filtering using matrix factorization
        - Matrix Factorization on only the observed items minimizes the objective function which is what we don't want
        - We can fix this using weighted-matrix-factorization, we treat unobserved entries as 0, but we also scale the un-observed part of the objective function
          so it is not over-weighted

# Building The Recommender System

In [3]:
import tensorflow_datasets as tfds
import tensorflow as tf
import numpy as np
import pandas as pd
import tensorflow_recommenders as tfrs

In [4]:
# Load movielens-ratings dataset
ratings = tfds.load("movie_lens/100k-ratings", split = "train")

2022-08-14 15:47:25.838186: W tensorflow/core/platform/cloud/google_auth_provider.cc:184] All attempts to get a Google authentication bearer token failed, returning an empty token. Retrieving token from files failed with "NOT_FOUND: Could not locate the credentials file.". Retrieving token from GCE failed with "FAILED_PRECONDITION: Error executing an HTTP request: libcurl code 6 meaning 'Couldn't resolve host name', error details: Could not resolve host: metadata".


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/100000 [00:00<?, ? examples/s]

Shuffling ~/tensorflow_datasets/movie_lens/100k-ratings/0.1.1.incompleteGS9JLG/movie_lens-train.tfrecord*...: …

Dataset movie_lens downloaded and prepared to ~/tensorflow_datasets/movie_lens/100k-ratings/0.1.1. Subsequent calls will reuse this data.


2022-08-14 15:48:11.260585: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
for x in ratings.take(1).as_numpy_iterator():
    print(x) 

{'bucketized_user_age': 45.0, 'movie_genres': array([7]), 'movie_id': b'357', 'movie_title': b"One Flew Over the Cuckoo's Nest (1975)", 'raw_user_age': 46.0, 'timestamp': 879024327, 'user_gender': True, 'user_id': b'138', 'user_occupation_label': 4, 'user_occupation_text': b'doctor', 'user_rating': 4.0, 'user_zip_code': b'53211'}


2022-08-14 15:48:16.146669: W tensorflow/core/kernels/data/cache_dataset_ops.cc:856] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


In [6]:
# Load in movielens-movies data
movies_df = tfds.load("movielens/100k-movies", split = "train")

for x in movies_df.take(1).as_numpy_iterator():
    print(x)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/1682 [00:00<?, ? examples/s]

Shuffling ~/tensorflow_datasets/movielens/100k-movies/0.1.1.incomplete648W4L/movielens-train.tfrecord*...:   0…

Dataset movielens downloaded and prepared to ~/tensorflow_datasets/movielens/100k-movies/0.1.1. Subsequent calls will reuse this data.
{'movie_genres': array([4]), 'movie_id': b'1681', 'movie_title': b'You So Crazy (1994)'}


2022-08-14 15:48:20.977753: W tensorflow/core/kernels/data/cache_dataset_ops.cc:856] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


In [7]:
ratings = ratings.map(lambda x: {"movie_title" : x["movie_title"],
                                 "user_id" : x["user_id"]}) # Lambda function that returns ratings as a MapDataset that has movie_title and user_id column.

In [8]:
movies = movies_df.map(lambda x: x["movie_title"])

In [9]:
user_ids_vocabulary = tf.keras.layers.StringLookup(mask_token=None) # user_ids_vocabulary is equal to a bunch of integers based off of a table-based vocabulary lookup
user_ids_vocabulary.adapt(ratings.map(lambda x: x["user_id"]))

In [ ]:
movie_titles_vocabulary = tf.keras.layers.StringLookup(mask_token = None)
movie_titles_vocabulary.adapt(movies)

In [ ]:
class MovieLensModel(tfrs.Model):
  def __init__(
      self,
      user_model: tf.keras.Model,
      movie_model: tf.keras.Model,
      task: tfrs.tasks.Retrieval): # what Retrieval does is that it retieves O(thousands) from a corpus of O(millions)
    
    super().__init__()

    # Set up user and movie representations
    self.user_model = user_model
    self.movie_model = movie_model
    # Set up retrieval task
    self.task = task
  
  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
      # Define how the loss is computed

      user_embeddings = self.user_model(features["user_id"])
      movie_embeddings = self.movie_model(features["movie_title"])

      return self.task(user_embeddings, movie_embeddings)

In [ ]:
user_model = tf.keras.Sequential([
    user_ids_vocabulary, # Keras preproccesing layer we could also do movie_titles_vocabulary = tf.keras.layers.StringLookup(vocabulary =  ratings, mask_token = None)
    tf.keras.layers.Embedding(user_ids_vocabulary.vocabulary_size(), 64)
])

movie_model = tf.keras.Sequential([
    movie_titles_vocabulary, 
    tf.keras.layers.Embedding(movie_titles_vocabulary.vocabulary_size(), 64)
])

task = tfrs.tasks.Retrieval(metrics = tfrs.metrics.FactorizedTopK(
    movies.batch(128).map(movie_model)
    )
)

In [ ]:
model = MovieLensModel(user_model, movie_model, task)
model.compile(optimizer = tf.keras.optimizers.Adagrad(0.5))

model.fit(ratings.batch(4096), epochs = 3)

In [ ]:
index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)

index.index_from_dataset(movies.batch(100).map(lambda title : (title, model.movie_model(title))))

_, titles = index(np.array(["138"]))
print(f"Top 3 recommendations for user 138: {titles[0, :3]}")

# Resources
        - Tensorflow Youtube Video's (https://www.youtube.com/watch?v=BthUPVwA59s&list=PLQY2H8rRoyvy2MiyUBz5RWZr5MPFkV3qz)
        - How to Design and Build a Recommendation System Pipeline in Python (Jill Cates) (https://www.youtube.com/watch?v=v_mONWiFv0k)
        - Building a Recommendation System in Python (https://www.youtube.com/watch?v=G4MBc40rQ2k)